# 🏆 Análisis del Mejor Escenario - Scenario_14

Este notebook analiza el **scenario_14**, que presenta la **mejor mejora en TMT** después de la optimización con algoritmo genético.

## 🎯 Resultados Destacados:
- **Mejora TMT**: +41.336s (de -45.681s a -4.346s)
- **Mejora Coordinación**: +84.0% (de 14.0% a 98.0%)
- **Ranking**: #1 entre los 68 escenarios optimizados

## Objetivos:
- Analizar las características que hicieron exitosa esta optimización
- Identificar patrones de configuración óptima
- Comparar con el scenario_2 problemático
- Proporcionar insights para mejorar otros escenarios


## 📊 Imports y Configuración


In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Configuración de matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
sns.set_palette("husl")

# Configuración de rutas
PROJECT_ROOT = Path.cwd().parent.parent if Path.cwd().name == 'notebooks' else Path.cwd().parent
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
RESULTS_DIR = PROJECT_ROOT / "results"

print(f"📁 Project root: {PROJECT_ROOT}")
print(f"📊 Data directory: {DATA_RAW}")
print(f"🔧 Processed directory: {DATA_PROCESSED}")
print(f"📈 Results directory: {RESULTS_DIR}")


📁 Project root: /Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG
📊 Data directory: /Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG/data/raw
🔧 Processed directory: /Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG/data/processed
📈 Results directory: /Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG/results


## 📊 Carga y Preparación de Datos


In [2]:
def load_json_data(file_path):
    """Load JSON data from file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def filter_scenario_data(data, scenario_id):
    """Filter data for specific scenario."""
    return [pair for pair in data if pair.get('scenario_id') == scenario_id]

def calculate_dt_values(pairs, cti=0.2):
    """Calculate DT values for all pairs."""
    dt_values = []
    pair_details = []
    
    for i, pair in enumerate(pairs):
        main_relay = pair.get('main_relay', {})
        backup_relay = pair.get('backup_relay', {})
        
        time_main = main_relay.get('Time_out', 0)
        time_backup = backup_relay.get('Time_out', 0)
        
        if isinstance(time_main, (int, float)) and isinstance(time_backup, (int, float)):
            dt = (time_backup - time_main) - cti
            dt_values.append(dt)
            
            pair_details.append({
                'pair_id': i + 1,
                'main_relay': main_relay.get('relay', 'Unknown'),
                'backup_relay': backup_relay.get('relay', 'Unknown'),
                'time_main': time_main,
                'time_backup': time_backup,
                'dt': dt,
                'coordinated': dt >= 0,
                'tds_main': main_relay.get('TDS', 0),
                'tds_backup': backup_relay.get('TDS', 0),
                'pickup_main': main_relay.get('pick_up', 0),
                'pickup_backup': backup_relay.get('pick_up', 0),
                'ishc_main': main_relay.get('Ishc', 0),
                'ishc_backup': backup_relay.get('Ishc', 0)
            })
    
    return dt_values, pair_details

# Cargar datos
print("📊 Cargando datos para scenario_14...")
original_data = load_json_data(DATA_RAW / "automation_results.json")
optimized_data = load_json_data(DATA_PROCESSED / "automation_results_scenario_14_optimized.json")

# Filtrar para scenario_14
original_pairs = filter_scenario_data(original_data, 'scenario_14')
optimized_pairs = filter_scenario_data(optimized_data, 'scenario_14')

print(f"✅ Datos originales: {len(original_pairs)} pares")
print(f"✅ Datos optimizados: {len(optimized_pairs)} pares")

# Calcular valores DT
original_dt, original_details = calculate_dt_values(original_pairs)
optimized_dt, optimized_details = calculate_dt_values(optimized_pairs)

print(f"✅ Valores DT calculados para ambos conjuntos de datos")


📊 Cargando datos para scenario_14...
✅ Datos originales: 100 pares
✅ Datos optimizados: 100 pares
✅ Valores DT calculados para ambos conjuntos de datos


## 📈 Métricas Principales - Scenario_14


In [3]:
def calculate_metrics(dt_values):
    """Calculate TMT and coordination metrics."""
    if not dt_values:
        return {
            'tmt': 0.0,
            'coordination_percentage': 0.0,
            'mean_dt': 0.0,
            'std_dt': 0.0,
            'min_dt': 0.0,
            'max_dt': 0.0
        }
    
    # Filtrar valores finitos para cálculos estadísticos
    finite_dt = [dt for dt in dt_values if np.isfinite(dt)]
    
    if not finite_dt:
        return {
            'tmt': 0.0,
            'coordination_percentage': 0.0,
            'mean_dt': 0.0,
            'std_dt': 0.0,
            'min_dt': 0.0,
            'max_dt': 0.0
        }
    
    tmt = sum(dt for dt in finite_dt if dt < 0)
    coordinated_pairs = sum(1 for dt in finite_dt if dt >= 0)
    coordination_percentage = (coordinated_pairs / len(finite_dt)) * 100
    
    return {
        'tmt': tmt,
        'coordination_percentage': coordination_percentage,
        'mean_dt': np.mean(finite_dt),
        'std_dt': np.std(finite_dt),
        'min_dt': np.min(finite_dt),
        'max_dt': np.max(finite_dt)
    }

# Calcular métricas
original_metrics = calculate_metrics(original_dt)
optimized_metrics = calculate_metrics(optimized_dt)

# Crear DataFrame de comparación
comparison_df = pd.DataFrame({
    'Métrica': ['TMT (s)', 'Coordinación (%)', 'DT Promedio (s)', 'DT Std (s)', 'DT Mín (s)', 'DT Máx (s)'],
    'Antes': [
        f"{original_metrics['tmt']:.3f}",
        f"{original_metrics['coordination_percentage']:.1f}",
        f"{original_metrics['mean_dt']:.3f}",
        f"{original_metrics['std_dt']:.3f}",
        f"{original_metrics['min_dt']:.3f}",
        f"{original_metrics['max_dt']:.3f}"
    ],
    'Después': [
        f"{optimized_metrics['tmt']:.3f}",
        f"{optimized_metrics['coordination_percentage']:.1f}",
        f"{optimized_metrics['mean_dt']:.3f}",
        f"{optimized_metrics['std_dt']:.3f}",
        f"{optimized_metrics['min_dt']:.3f}",
        f"{optimized_metrics['max_dt']:.3f}"
    ]
})

print("🏆 MÉTRICAS PRINCIPALES - SCENARIO_14 (MEJOR PERFORMANCE)")
print("=" * 65)
print(comparison_df.to_string(index=False))

# Calcular mejoras
tmt_improvement = optimized_metrics['tmt'] - original_metrics['tmt']
coord_improvement = optimized_metrics['coordination_percentage'] - original_metrics['coordination_percentage']

print(f"\n🎯 MEJORAS EXCEPCIONALES:")
print(f"  TMT: {tmt_improvement:+.3f}s (¡MEJOR DE TODOS LOS ESCENARIOS!)")
print(f"  Coordinación: {coord_improvement:+.1f}%")

# Comparar con scenario_2
print(f"\n📊 COMPARACIÓN CON SCENARIO_2:")
print(f"  Scenario_14 TMT Improvement: {tmt_improvement:+.3f}s")
print(f"  Scenario_2 TMT Improvement: +12.003s")
print(f"  Diferencia: {tmt_improvement - 12.003:+.3f}s (Scenario_14 es {abs(tmt_improvement - 12.003):.3f}s mejor)")


🏆 MÉTRICAS PRINCIPALES - SCENARIO_14 (MEJOR PERFORMANCE)
         Métrica   Antes Después
         TMT (s) -45.681  -4.346
Coordinación (%)    14.0    98.0
 DT Promedio (s)  -0.308   1.455
      DT Std (s)   2.222   4.806
      DT Mín (s) -14.306  -2.370
      DT Máx (s)  13.920  45.964

🎯 MEJORAS EXCEPCIONALES:
  TMT: +41.336s (¡MEJOR DE TODOS LOS ESCENARIOS!)
  Coordinación: +84.0%

📊 COMPARACIÓN CON SCENARIO_2:
  Scenario_14 TMT Improvement: +41.336s
  Scenario_2 TMT Improvement: +12.003s
  Diferencia: +29.333s (Scenario_14 es 29.333s mejor)
